In [3]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import difflib #it will find the matchest one
import requests

In [4]:
data1 = pd.read_csv('/content/drive/MyDrive/Data science/Projects/Movie Recomendation/Movies Data/IMDB-Movie-Data.csv')
data2 = pd.read_csv('/content/drive/MyDrive/Data science/Projects/Movie Recomendation/Movies Data/TMDB.csv')
data3 = pd.read_csv('/content/drive/MyDrive/Data science/Projects/Movie Recomendation/Movies Data/movies.csv')
data4 = pd.read_csv('/content/drive/MyDrive/Data science/Projects/Movie Recomendation/Movies Data/mymoviedb.csv',lineterminator='\n')
data5 = pd.read_csv('/content/drive/MyDrive/Data science/Projects/Movie Recomendation/Movies Data/IMDb_All_Genres_etf_clean1.csv')

#Data 1

In [5]:
data1 = data1.drop(['Metascore','Revenue (Millions)','Votes','Rating','Runtime (Minutes)','Description','Rank'],axis=1)

In [6]:
data1.head()

,Title,Genre,Director,Actors,Year
0,Guardians of the Galaxy,"Action,Adventure,Sci-Fi",James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...",2014
1,Prometheus,"Adventure,Mystery,Sci-Fi",Ridley Scott,"Noomi Rapace, Logan Marshall-Green, Michael Fa...",2012
2,Split,"Horror,Thriller",M. Night Shyamalan,"James McAvoy, Anya Taylor-Joy, Haley Lu Richar...",2016
3,Sing,"Animation,Comedy,Family",Christophe Lourdelet,"Matthew McConaughey,Reese Witherspoon, Seth Ma...",2016
4,Suicide Squad,"Action,Adventure,Fantasy",David Ayer,"Will Smith, Jared Leto, Margot Robbie, Viola D...",2016


In [7]:
data1.shape

(1000, 5)

In [8]:
data1.isnull().any()

Title       False
Genre       False
Director    False
Actors      False
Year        False
dtype: bool

In [9]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Title     1000 non-null   object
 1   Genre     1000 non-null   object
 2   Director  1000 non-null   object
 3   Actors    1000 non-null   object
 4   Year      1000 non-null   int64 
dtypes: int64(1), object(4)
memory usage: 39.2+ KB


#**Data 2**

In [10]:
#Deleting unwanted columns
data2 = data2.drop(['crew','cast','poster_path','adult_tf','popularity','vote_average','vote_count','budget','runtime','tagline','status','revenue','production_companies','spoken_languages','original_title','keywords','overview','movie_id','Unnamed: 0'],axis=1)

In [11]:
# Here we need to clean the genres column but those all are in type(str), so we cant use replace method

#adding aa data in genre to a list
genre = []

for i in data2['genres']:
  genre.append(i)

In [12]:
#removing all unwanted characters, numerics and others from the list made above

genre = [re.sub(r'[^\w\s]', '', x) for x in genre]
genre = [re.sub(r'\d', '', x) for x in genre]
genre = [re.sub('name', '', x) for x in genre]
genre = [re.sub('id', '', x) for x in genre]
genre = [x.replace("'", "").replace('"', "") for x in genre]

In [13]:
#Assining this list to the dataframe

data2['genre'] = genre

In [14]:
# adding commas to between genres

data2['genre'] = data2['genre'].str.replace('   ',',')
data2['genre'] = data2['genre'].str[0:] #to replace the comma at first

In [15]:
#removing old genre column

data2 = data2.drop(['genres'],axis = 1)

In [16]:
#removing date and month from the column release date
data2['release_date'] =  pd.to_datetime(data2['release_date'])
data2['release_date'] = data2['release_date'].dt.strftime('%Y')


In [17]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   title              10000 non-null  object
 1   release_date       9977 non-null   object
 2   original_language  10000 non-null  object
 3   genre              10000 non-null  object
dtypes: object(4)
memory usage: 312.6+ KB


In [18]:
data2.head()

,title,release_date,original_language,genre
0,Avatar: The Way of Water,2022,en,",Science Fiction, Adventure, Action"
1,Glass Onion: A Knives Out Mystery,2022,en,",Mystery, Thriller, Comedy"
2,Violent Night,2022,en,",Action, Comedy, Crime, Thriller"
3,"The Chronicles of Narnia: The Lion, the Witch ...",2005,en,",Adventure, Family, Fantasy"
4,Black Adam,2022,en,",Action, Fantasy, Science Fiction"


#**Data 3**

In [19]:
#removing unwnted columns
data3 = data3.drop(['index','budget','homepage','id','overview','original_title','popularity','runtime','spoken_languages','status','tagline','vote_average','vote_count','keywords','production_companies','crew','production_countries','revenue'],axis=1)

In [20]:
data3.head()

,genres,original_language,release_date,title,cast,director
0,Action Adventure Fantasy Science Fiction,en,2009-12-10,Avatar,Sam Worthington Zoe Saldana Sigourney Weaver S...,James Cameron
1,Adventure Fantasy Action,en,2007-05-19,Pirates of the Caribbean: At World's End,Johnny Depp Orlando Bloom Keira Knightley Stel...,Gore Verbinski
2,Action Adventure Crime,en,2015-10-26,Spectre,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,Sam Mendes
3,Action Crime Drama Thriller,en,2012-07-16,The Dark Knight Rises,Christian Bale Michael Caine Gary Oldman Anne ...,Christopher Nolan
4,Action Adventure Science Fiction,en,2012-03-07,John Carter,Taylor Kitsch Lynn Collins Samantha Morton Wil...,Andrew Stanton


In [21]:
#changging date to Datetime
data3['release_date'] = pd.to_datetime(data3['release_date'])

#picking only year from the data we just need only year for similarities
data3['release_date'] = data3['release_date'].dt.strftime('%Y')

In [22]:
data3.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   genres             4775 non-null   object
 1   original_language  4803 non-null   object
 2   release_date       4802 non-null   object
 3   title              4803 non-null   object
 4   cast               4760 non-null   object
 5   director           4773 non-null   object
dtypes: object(6)
memory usage: 225.3+ KB


#Data4

In [23]:
#removing unwanted columns

data4 = data4.drop(['Overview','Popularity','Vote_Count','Vote_Average','Poster_Url'],axis=1)

In [24]:
#converting relase date to datetime
data4['Release_Date'] = pd.to_datetime(data4['Release_Date'])

#remaining only year from date column
data4['Release_Date'] = data4['Release_Date'].dt.strftime('%Y')

In [25]:
data4.head()

,Release_Date,Title,Original_Language,Genre
0,2021,Spider-Man: No Way Home,en,"Action, Adventure, Science Fiction"
1,2022,The Batman,en,"Crime, Mystery, Thriller"
2,2022,No Exit,en,Thriller
3,2021,Encanto,en,"Animation, Comedy, Family, Fantasy"
4,2021,The King's Man,en,"Action, Adventure, Thriller, War"


#**Data 5**

In [26]:
data5.head()

,Movie_Title,Year,Director,Actors,Rating,Runtime(Mins),Censor,Total_Gross,main_genre,side_genre
0,Kantara,2022,Rishab Shetty,"Rishab Shetty, Sapthami Gowda, Kishore Kumar G...",9.3,148,UA,Gross Unkown,Action,"Adventure, Drama"
1,The Dark Knight,2008,Christopher Nolan,"Christian Bale, Heath Ledger, Aaron Eckhart, M...",9.0,152,UA,$534.86M,Action,"Crime, Drama"
2,The Lord of the Rings: The Return of the King,2003,Peter Jackson,"Elijah Wood, Viggo Mortensen, Ian McKellen, Or...",9.0,201,U,$377.85M,Action,"Adventure, Drama"
3,Inception,2010,Christopher Nolan,"Leonardo DiCaprio, Joseph Gordon-Levitt, Ellio...",8.8,148,UA,$292.58M,Action,"Adventure, Sci-Fi"
4,The Lord of the Rings: The Two Towers,2002,Peter Jackson,"Elijah Wood, Ian McKellen, Viggo Mortensen, Or...",8.8,179,UA,$342.55M,Action,"Adventure, Drama"


In [27]:
#here i want to combine main_genre column and side_genre column
data5['Genre'] = data5['main_genre'].str.cat(data5['side_genre'],sep=',')

#there are repeted values in our combined column we need to remove it
data5.drop_duplicates(subset='Genre',keep='first',inplace=True)

In [28]:
#removing unwanted columns
data5 = data5.drop(['Rating','Runtime(Mins)','Censor','Total_Gross','main_genre','side_genre'],axis=1)

In [29]:
data5.head()

,Movie_Title,Year,Director,Actors,Genre
0,Kantara,2022,Rishab Shetty,"Rishab Shetty, Sapthami Gowda, Kishore Kumar G...","Action, Adventure, Drama"
1,The Dark Knight,2008,Christopher Nolan,"Christian Bale, Heath Ledger, Aaron Eckhart, M...","Action, Crime, Drama"
3,Inception,2010,Christopher Nolan,"Leonardo DiCaprio, Joseph Gordon-Levitt, Ellio...","Action, Adventure, Sci-Fi"
6,The Matrix,1999,"Directors:Lana Wachowski, Lilly Wachowski","Keanu Reeves, Laurence Fishburne, Carrie-Anne ...","Action, Sci-Fi"
7,The Empire Strikes Back,1980,Irvin Kershner,"Mark Hamill, Harrison Ford, Carrie Fisher, Bil...","Action, Adventure, Fantasy"


#creating actual dataframe

In [30]:
#in this data3 we have 6 columns so we have to add on other dataframes

data2[['Director','Actors']] = None
data1['Language'] = None
data4['Actors','Director'] = None
data5['Language'] = None


In [31]:
#changing the oder of the data frmes by column
data1 = data1.reindex(columns=['Title','Genre','Language','Year','Actors','Director'])
data2 = data2.reindex(columns=['title','genre','language','release_date','Actors','Director'])
data3 = data3.reindex(columns=['title','genres','original_language','release_date','cast','director'])
data4 = data4.reindex(columns=['Title','Genre','Original_Language','Release_Date','Actors','Director',])
data5 = data5.reindex(columns=['Movie_Title','Genre','Language','Year','Actors','Director'])

In [32]:
#we need to make all the column headers same name to concat otherwise it will add new columnn

data2.rename(columns={'title':'Title','genre':'Genre','language':'Language','release_date':'Year','cast':'Actors','director':'Director'},inplace = True)
data3.rename(columns={'title':'Title','genres':'Genre','original_language':'Language','release_date':'Year','cast':'Actors','director':'Director'},inplace = True)
data4.rename(columns={'title':'Title','Original_Language':'Language','Release_Date':'Year'},inplace=True)
data5.rename(columns={'Movie_Title':'Title'},inplace=True)

In [33]:
#adding these columns
movies = pd.concat([data1,data2,data3,data4,data5])

#**-------------------------------------------------------**

**Here** we need the delet same like movies from the dataset we cant use drop_duplicate function bcs data in their row by tittle is same but other columns are not,we need to delet and remain one data of movie row which gives more informations or vectors

In [34]:
movies.shape

(25956, 6)

In [35]:
#creating a dataframe  without titie column
features = movies.drop('Title',axis=1)

In [36]:
#getiing the number of vectors/informations/letters by each row excluding NUN or null values and adding as a column
movies['len']= features.applymap(lambda x: len(str(x)) if pd.notnull(x) else 0).sum(axis=1)



In [37]:
#sorting all dataset which has maximum number of words in each row
movies = movies.sort_values(by='len',ascending=False)


In [38]:
#removing every duplicate rows which has same value on titile column and keep the first one(whivh has maximum number of strings)
movies.drop_duplicates(subset='Title',keep='first',inplace=True)

In [39]:
#droping length column

movies = movies.drop(['len'],axis=1)

In [40]:
#removing rows wich has missing values
movies = movies.dropna(thresh=movies.shape[1]-2)

In [41]:
movies.shape

(11440, 6)

In [42]:
# # #filling missing vlues that we created empty columns with NAN

movies['Language'].fillna(value = 'NaN',inplace = True)
movies['Actors'].fillna(value = 'NaN',inplace = True)
movies['Director'].fillna(value = 'NaN',inplace = True)
movies['Genre'].fillna(value = 'NaN',inplace = True)
movies['Year'].fillna(value = 'NaN',inplace = True)

In [43]:
# Adding a new column for index
movies.reset_index(drop=False, inplace=True)

In [44]:
movies.isnull().sum()

index       0
Title       0
Genre       0
Language    0
Year        0
Actors      0
Director    0
dtype: int64

#**----------------------------------------------------------------**

In [45]:
# movies['Year'] = pd.to_datetime(movies['Year'])

In [46]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11440 entries, 0 to 11439
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   index     11440 non-null  int64 
 1   Title     11440 non-null  object
 2   Genre     11440 non-null  object
 3   Language  11440 non-null  object
 4   Year      11440 non-null  object
 5   Actors    11440 non-null  object
 6   Director  11440 non-null  object
dtypes: int64(1), object(6)
memory usage: 625.8+ KB


#Creating predictibve system

In [47]:
#Concating featrues of movies into one colomn
features = movies['Genre']+' '+movies['Language']+' '+movies['Year'].astype(str)+' '+movies['Actors']+' '+movies['Director']

In [48]:
#Vectorizing all the features we want

vectorizer = TfidfVectorizer()
vector = vectorizer.fit_transform(features)
cosine = cosine_similarity(vector)



##**Setting up a movie predictive sysytem**

In [49]:
#convertinf movie titiles to a list
title_list = movies['Title'].tolist()



In [50]:
movie_name = input('enter your movie: ')


enter your movie: sd


In [51]:
#getting clossest input from the movie list using difflib
matched = difflib.get_close_matches(movie_name,title_list)
matched

[]

In [52]:
#choosing the first movie from suggestions
pick = matched[0]
pick

IndexError: ignored

In [ ]:
#getting the index of input movie
index = movies.index[movies['Title'] == pick][0]
index

In [ ]:
#getting a list of smilar movies
similar_movies = list(enumerate(cosine[index]))
# print(similar_movies) #index,similarscore

In [ ]:
#arranging movies by similarity score by the given movie
top_movies = sorted(similar_movies,key = lambda x:x[1],reverse = True)
# print(top_movies)

In [ ]:
#getting name from index

print('Movies suggested for you : \n')

i = 1

for movie in top_movies:
  index = movie[0]
  title_from_index = movies.iloc[index]['Title']
  if (i < 10):
    print(i, '.', title_from_index)
    i += 1

In [ ]:
movies.to_csv('modified_csv_file.csv', index=False)

In [ ]:

rslt = []

x = 1


for i in top_movies:
    x += 1
    if x <= 12:
      rslt.append(i)

otp_lst = []
for x,y in rslt:
  otp_lst.append(x)


otp = movies['Title'].loc[otp_lst]
otp.reset_index(drop=True)

#**Telegram bot**

In [ ]:
last_update_id = 0

while True:

    base = 'https://api.telegram.org/bot{API KEY}'

    get_params = {
        'offset': last_update_id,
        'timeout': 30  # Increase the timeout to avoid frequent polling
    }

    req = requests.get(base + '/getUpdates', params=get_params)

    data = req.json()

    # Check if there are any updates
    if len(data['result']) > 0:

        # Get the latest update from the data
        update = data['result'][-1]

        # Get the chat id and text from the latest update
        chat = update['message']['chat']['id']
        inp = update['message'].get('text', '')

        # Load the movies data
        movies = pd.read_csv('/content/drive/MyDrive/Data science/Projects/Movie Recomendation/Movies Data/Shiyas Movies.csv')

        # Concatenate the features of movies into one column
        features = movies['Genre'] + ' ' + movies['Language'] + ' ' + movies['Year'].astype(str) + ' ' + movies['Actors'] + ' ' + movies['Director']

        # Vectorize all the features we want
        vectorizer = TfidfVectorizer()
        vector = vectorizer.fit_transform(features)
        cosine = cosine_similarity(vector)

        # Setting up a movie predictive system
        # Converting movie titles to a list
        title_list = movies['Title'].tolist()

        movie_name = inp

        # Getting closest input from the movie list using difflib
        matched = difflib.get_close_matches(movie_name, title_list)
        if matched:
            # Choosing the first movie from suggestions
            pick = matched[0]

            # Getting the index of input movie
            index = movies.index[movies['Title'] == pick][0]

            # Getting a list of similar movies
            similar_movies = list(enumerate(cosine[index]))

            # Arranging movies by similarity score by the given movie
            top_movies = sorted(similar_movies, key=lambda x: x[1], reverse=True)

            # Getting suggested movies from top_movies
            rslt = []

            x = 1

            # Iterating first 10 result from top movies
            for i in top_movies:
                x += 1
                if x <= 12:
                    rslt.append(i)

            # Removing similarity score from the list
            otp_lst = []
            for x, y in rslt:
                otp_lst.append(x)

            # Getting name of the movies using result as index
            otp = movies['Title'].loc[otp_lst]

            # Removing old index from output and setting new index
            otp = otp.reset_index(drop=True)
            otp = '\n'.join(otp)  # Telegram doesn't send a series so convert into string

            # Sending the output
            prm = {
                'chat_id': chat,
                'text': otp,
                'allow_sending_without_reply':False

            }
            # to give welcome message
            if inp == "/start":
              prm1 = {
                  'chat_id' : chat,
                  'text' : "Welcome to Shiyas Movie Recomendation system.give me an example of movie that you want to watch",
                  'allow_sending_without_reply':False
              }
              get1 = requests.get(base + '/sendMessage', params = prm1)
            else:
               get = requests.get(base + '/sendMessage', params = prm)

        # Update the last update id to the latest update id + 1
        last_update_id = update['update_id'] + 1

        print(req.text)

{"ok":true,"result":[{"update_id":733828932,
"message":{"message_id":5,"from":{"id":747848474,"is_bot":false,"first_name":"Shiyas","last_name":"Ali","username":"Shiyas_ali","language_code":"en"},"chat":{"id":747848474,"first_name":"Shiyas","last_name":"Ali","username":"Shiyas_ali","type":"private"},"date":1690986082,"text":"start"}}]}
{"ok":true,"result":[{"update_id":733828933,
"message":{"message_id":7,"from":{"id":5021183499,"is_bot":false,"first_name":"dh","language_code":"en"},"chat":{"id":5021183499,"first_name":"dh","type":"private"},"date":1690986235,"text":"/start","entities":[{"offset":0,"length":6,"type":"bot_command"}]}}]}
{"ok":true,"result":[{"update_id":733828934,
"message":{"message_id":9,"from":{"id":5021183499,"is_bot":false,"first_name":"dh","language_code":"en"},"chat":{"id":5021183499,"first_name":"dh","type":"private"},"date":1690986263,"text":"fifty shades of grey"}}]}
